# 📌 Extracción
Se descargan los datos de la API JSON de TelecomX y se aplanan usando `json_normalize`.

In [ ]:
import pandas as pd
from pandas import json_normalize
import requests

# Descargar datos
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/main/TelecomX_Data.json'
data = requests.get(url).json()

# Flatten del JSON
df = json_normalize(data, sep='_')
df.head()

# 🔧 Transformación
Se limpian columnas problemáticas, se normalizan nombres, se convierten tipos y se rellenan nulos.

In [ ]:
import numpy as np

# Eliminar columnas con dict o list
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, (dict,list))).any():
        df.drop(col, axis=1, inplace=True)

# Normalizar nombres
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Convertir columnas numéricas si existen
for col in ['tenure','monthly','total']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Churn a 0/1
if 'churn' in df.columns:
    df['churn'] = df['churn'].map({'Yes':1,'No':0})

# Rellenar nulos
num_cols = df.select_dtypes(include='number').columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols] = df[cat_cols].fillna('Desconocido')

df.head()

# 📊 Carga y análisis
Se realiza análisis exploratorio de datos con gráficos y correlación.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Distribución de churn
plt.figure(figsize=(6,4))
sns.countplot(x='churn', data=df, palette='Set2')
plt.title('Distribución de Churn')
plt.show()

# Pie chart de churn
if 'churn' in df.columns:
    churn_counts = df['churn'].value_counts()
    labels = ['Se quedan','Se van']
    colors = ['#8BC34A','#FF5252']
    plt.figure(figsize=(6,6))
    plt.pie(churn_counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors, explode=(0,0.1))
    plt.title('Proporción de clientes que se quedan vs que se van')
    plt.show()

# Correlación entre numéricas
num_for_corr = [c for c in ['tenure','monthly','total','churn'] if c in df.columns]
if len(num_for_corr) > 1:
    plt.figure(figsize=(6,5))
    sns.heatmap(df[num_for_corr].corr(), annot=True, cmap='coolwarm')
    plt.title('Mapa de correlación')
    plt.show()

# 📄 Informe final
Conclusiones y recomendaciones basadas en el análisis de Churn.

In [ ]:
print("📌 Conclusiones del análisis:\n" +
      f"- La tasa de churn es aproximadamente {df['churn'].mean()*100:.2f}%\n" +
      "- Contratos mensuales muestran mayor tendencia a cancelación.\n" +
      "- Clientes con Fibra óptica tienden a irse más que DSL.\n" +
      "- Clientes nuevos (menos de 1 año de antigüedad) tienen mayor riesgo de churn.\n" +
      "- Clientes con facturas mensuales altas se van más que los de menor gasto.\n" +
      "\n✅ Recomendaciones:\n" +
      "- Incentivar contratos a largo plazo.\n" +
      "- Ofrecer planes competitivos de fibra óptica.\n" +
      "- Diseñar programas de retención para clientes nuevos y de alto gasto.")